In [7]:
## Questions

Are students asking conceptual or appllied questions?
Which topics are more prevalent? Where is the most confusion?
    - Topic Modeling
    - Run the same topic model over different intervals of time
What do in-person interactions not already tell us? 
What are areas that need attention in student understanding? 
    - Concrete evidence of topics where students are struggling
Probably don't look at response time
What are people afraid of asking about? (Anonymous posts from students)
Data:
    - Timestamp
    - Userid (student or instructor, throw out announcements)
    - Question or answer? 
    - Votes (good question)
    - Question Text

    
diff values of config?
Check status = active
transform to get length of upvote_ids

Question or Answer? 
Num votes/good question
where does anon equal yes

IndentationError: unexpected indent (<ipython-input-7-3815f359c888>, line 5)

In [1]:
import glob
import pickle
import pandas as pd
import numpy as np
import scipy
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.decomposition import LatentDirichletAllocation 

In [2]:
dfs = glob.glob('data/dataframes/*.p')
print(dfs[0])

data/dataframes/kimia@live.unc.edu_dataframe_jkws0l0gvcr7it.p


In [3]:
textList = []

currentDf = pd.read_pickle(dfs[0])
dataList = currentDf["text"].tolist()

In [21]:
from IPython.display import display
def print_topics(model, vectorizer, top_n=10): 
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

num_features = 400
tf_vectorizer = CountVectorizer(max_df=.85, min_df=.05, max_features=num_features, stop_words='english', token_pattern=u'(?ui)\\b\\w\w*[a-z]+\\w*\\b')

ldaList = []

data_samples = dataList
tf_data_samples = tf_vectorizer.fit_transform(data_samples) 
tf_feature_names = tf_vectorizer.get_feature_names()

num_topics = 6
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=100, learning_method='online', learning_offset=10.,random_state=1).fit(tf_data_samples)
lda.score(tf_data_samples)

ldaList.append(lda)

print_topics(lda,tf_vectorizer)

Topic 0:
[('local', 12.8595095007747), ('checks', 9.991113394924758), ('s', 8.967966017290717), ('t', 8.274732618481714), ('work', 5.370342900381665), ('time', 5.286301646155675), ('check', 5.118068369539516), ('questions', 5.05943331314642), ('grader', 4.404751561057487), ('sure', 4.3634934549014215)]
Topic 1:
[('class', 35.09672914055233), ('t', 35.00374130169224), ('assignment', 17.94454387894138), ('getting', 17.20939808702263), ('don', 17.19033861220287), ('server', 17.15230354430696), ('m', 17.021438608015675), ('test', 17.013806146053092), ('grading', 16.05940882953443), ('method', 15.86109092926796)]
Topic 2:
[('class', 43.344777277799274), ('exam', 32.752105805505074), ('credit', 21.582961672054786), ('extra', 19.88113933285889), ('make', 16.065618336085805), ('points', 14.240048900734557), ('sure', 14.084952795895628), ('time', 14.046774475734525), ('prasun', 13.959048217643337), ('know', 13.143541548703384)]
Topic 3:
[('assignment', 29.724527654659028), ('s', 21.942620197462